In [8]:
import os
import torch
import cv2 as cv
import numpy as np
import torchvision
from torchvision import transforms
import pandas as pd

In [9]:
# case can have to be mix of this two groops (rgb, dept, Ergbd, Lrgbd) and (rad, angle)

case = 'Lrgbd_angle'
ver = 0

In [10]:
class LateFusionNet(torch.nn.Module):
    def __init__(self):
        super(LateFusionNet, self).__init__()
        # Separate convolutional layers for RGB and depth streams
        self.conv_rgb = torchvision.models.resnet18(pretrained=False)
        self.conv_rgb.fc = torch.nn.Identity()
        
        self.conv_depth = torchvision.models.resnet18(pretrained=False)
        self.conv_depth.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.conv_depth.fc = torch.nn.Identity()
        
        # Fully connected layers for fusion
        self.fc1 = torch.nn.Linear(512 * 2, 128)  # Concatenating features from both streams
        self.fc2 = torch.nn.Linear(128, 1)  # Adjust num_classes based on your task
        
    def forward(self, rgbd_inputs):
        rgb_inputs = rgbd_inputs[:,:3,:,:]
        depth_inputs = rgbd_inputs[:,3,:,:].unsqueeze(1)

        # Forward pass for RGB stream
        rgb_features = self.conv_rgb(rgb_inputs)
        # Forward pass for depth stream
        depth_features = self.conv_depth(depth_inputs)
        
        # Concatenate features from both streams
        combined_features = torch.cat((rgb_features, depth_features), dim=1)
        
        # Fully connected layers for fusion
        x = self.fc1(combined_features)
        x = torch.relu(x)
        x = self.fc2(x)
        return x


In [11]:
rgb_img_dir = './dataset/test/rgb/'
depth_img_dir = './dataset/test/depth/'

csv_path = './dataset/test/test_data.csv'

In [12]:
# Loads model.
def model_definition(weights, case):
    
    model = torchvision.models.resnet18(pretrained=False)
    model.fc = torch.nn.Linear(512, 1)

    if case.split('_')[0] == 'depth':
        model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    # elif case.split('_')[0] == 'rgb':
    # model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
    elif case.split('_')[0] == 'Ergbd':
        model.conv1 = torch.nn.Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    elif case.split('_')[0] == 'Lrgbd':
        model = LateFusionNet()
        
    model.eval()
    model.load_state_dict(torch.load(weights)) # Load weights
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    return model, device

# transfer image to tensor.
def im2tens(rgb, depth, case):
        
        rgb_transform = torchvision.transforms.ToTensor()
        depth_transform = torchvision.transforms.Compose([transforms.ToTensor(), transforms.Grayscale(num_output_channels=1)])
                 
        rgb_tensor = rgb_transform(rgb)
        depth_tensor = depth_transform(depth)

        rgbd_tensor = torch.cat([rgb_tensor, depth_tensor], dim=0)
        
        if case.split('_')[0] == 'rgb':
                img_tensor = rgb_tensor
                
        elif case.split('_')[0] == 'depth':
                img_tensor = depth_tensor
        else:
                img_tensor = rgbd_tensor

        return img_tensor


# Pregict class.
def predict(img, model, device):
    
    xb = img.unsqueeze(0)
    yb = model(xb.to(device))
    dist = yb.cpu().detach().numpy()[0][0]
    
    return dist
            
def get_models(rad_weights, angle_weights, case):
    
    rad_model, device = model_definition(rad_weights, case)
    angle_model, _     = model_definition(angle_weights, case)

    return rad_model, angle_model, device

def main_pred(rgb_img, depth_img, rad_model, angle_model, device, case):
    
    tensor_img = im2tens(rgb_img, depth_img, case)
    rad_pred = predict(tensor_img, rad_model, device)
    angle_pred = predict(tensor_img, angle_model, device)
    
    return rad_pred, angle_pred

In [13]:
rad_weights  = './Weights/' + case.split('_')[0] + '_rad_v' + str(ver) + '.pth'
angle_weights = './Weights/' + case.split('_')[0] + '_angle_v' + str(ver) + '.pth'

rad_model, angle_model, device = get_models(rad_weights, angle_weights, case)

In [14]:
output_dir = './results/' + case.split('_')[0] + '_v' + str(ver) + '/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_csv_path = os.path.join(output_dir, 'graphs_data.csv')

df = pd.read_csv(csv_path)
im_name = df.loc[:, 'Picture name']
angle = df.loc[:, 'Angle [deg]']
rad = df.loc[:, 'Radial distance [mm]']

data = []
        
for ii in range(len(im_name)):
    
    rgb_img = cv.cvtColor(cv.imread(rgb_img_dir + im_name[ii]), cv.COLOR_BGR2RGB)
    depth_img = cv.cvtColor(cv.imread(depth_img_dir + im_name[ii]), cv.COLOR_BGR2RGB)
    rad_pred, angle_pred = main_pred(rgb_img, depth_img, rad_model, angle_model, device, case)    
         
    if np.abs(angle_pred - angle[ii]) > 180:
        angle_pred = 360 - angle_pred
        
    data.append({
        'Picture name': im_name[ii],
        'gt_rad': rad[ii],
        'pred_rad': rad_pred,
        'gt_angle': angle[ii],
        'pred_angle': angle_pred
    })
    
# Convert the list of dictionaries to a DataFrame
data_df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
data_df.to_csv(output_csv_path, index=False)